* dm04 演習テーマ１ 埼玉大学の電力使用量の分析
  * 2025-のデータ(新フォーマット)の前処理、集約

（新フォーマット）
* 全体 / 日報1 (4,20)-(4,43)
* 電気電子3変電所 / 日報2 (4,20)-(4,43)
* 機械工学変電所 / 日報2 (5,20)-(5,43)
* デザイン1号変電所 / 日報2 (12,20)-(12,43)
* 応用化学変電所 / 日報2 (13,20)-(13,43)
* 事務局棟変電所 / 日報2 (14,20)-(14,43)
* イノベーション変電所 / 日報2 (15,20)-(15,43)
* 教養学部変電所 / 日報2 (16,20)-(16,43)
* 理学部3号変電所 / 日報3 (13,20)-(13,43)
* 理学部2号変電所 / 日報3 (14,20)-(14,43)
* 電気電子1変電所 / 日報3 (17,20)-(17,43)
* 情報工学科変電所 / 日報3 (19,20)-(19,43)
* 分析センター変電所 / 日報4 (4,20)-(4,43)
* 大学会館変電所 / 日報4 (10,20)-(10,43)
* 図書館変電所 / 日報4 (18,20)-(18,43)
* 廃液処理変電所 / 日報4 (19,20)-(19,43)
* 教育学部C変電所 / 日報5 (4,20)-(4,43)
* 総合体育館変電所 / 日報5 (8,20)-(8,43)
* 経済学部変電所 / 日報5 (9,20)-(9,43)
* 総合研究棟変電所 / 日報5 (11,20)-(11,43)
    
（旧フォーマット）
* 全体 / 日報
* ⑪機械棟変電所 / サブ変電所日報（１）
* ⑬機能材料変電所 / サブ変電所日報（１）
* ⑮建設工学１号館変電所 / サブ変電所日報（２）
* ⑭応用化学２号館変電所 / サブ変電所日報（２）
* ①本部変電所 / サブ変電所日報（２）
* ⑯地域共同変電所 / サブ変電所日報（２）
* ③教養学部変電所 / サブ変電所日報（３）
* ⑤理学部３号館変電所 / サブ変電所日報（４）
* ⑥理学部２号館変電所 / サブ変電所日報（４）
* ⑨電気電子１号変電所 / サブ変電所日報（５）
* ⑩情報工学科棟変電所 / サブ変電所日報（５）
* ⑧情報分析センタ－変電所 / サブ変電所日報（５）
* ②大学会館変電所 / サブ変電所日報（６）
* ⑦図書館変電所 / サブ変電所日報（６）
* ⑳廃液処理施設変電所 / サブ変電所日報（７）
* ⑰教育学部変電所 / サブ変電所日報（７）
* ⑱体育施設変電所 / サブ変電所日報（７）
* ④経済学部変電所 / サブ変電所日報（８）
* ⑫総合研究棟変電所 / サブ変電所日報（８）


In [1]:
import os
import sys
import re
import glob
import pandas as pd
import openpyxl # .xlsx ファイルの操作
import xlrd # .xls ファイルの読み込み

In [2]:
data_location = { # セル位置は、(行,列)で指定 1始まり
    '全体': {'sheetname': '日報1', 
           'data': {1:(20,4), 2:(21,4), 3:(22,4), 4:(23,4), 5:(24,4), 6:(25,4), 
                    7:(26,4), 8:(27,4), 9:(28,4), 10:(29,4), 11:(30,4), 12:(31,4), 
                    13:(32,4), 14:(33,4), 15:(34,4), 16:(35,4), 17:(36,4), 18:(37,4), 
                    19:(38,4), 20:(39,4), 21:(40,4), 22:(41,4), 23:(42,4), 24:(43,4),}}, 
    '電気電子3変電所': {'sheetname': '日報2', 
           'data': {1:(20,4), 2:(21,4), 3:(22,4), 4:(23,4), 5:(24,4), 6:(25,4), 
                    7:(26,4), 8:(27,4), 9:(28,4), 10:(29,4), 11:(30,4), 12:(31,4), 
                    13:(32,4), 14:(33,4), 15:(34,4), 16:(35,4), 17:(36,4), 18:(37,4), 
                    19:(38,4), 20:(39,4), 21:(40,4), 22:(41,4), 23:(42,4), 24:(43,4),}}, 
    '機械工学変電所': {'sheetname': '日報2', 
           'data': {1:(20,5), 2:(21,5), 3:(22,5), 4:(23,5), 5:(24,5), 6:(25,5), 
                    7:(26,5), 8:(27,5), 9:(28,5), 10:(29,5), 11:(30,5), 12:(31,5), 
                    13:(32,5), 14:(33,5), 15:(34,5), 16:(35,5), 17:(36,5), 18:(37,5), 
                    19:(38,5), 20:(39,5), 21:(40,5), 22:(41,5), 23:(42,5), 24:(43,5),}}, 
    'デザイン1号変電所': {'sheetname': '日報2', 
           'data': {1:(20,12), 2:(21,12), 3:(22,12), 4:(23,12), 5:(24,12), 6:(25,12), 
                    7:(26,12), 8:(27,12), 9:(28,12), 10:(29,12), 11:(30,12), 12:(31,12), 
                    13:(32,12), 14:(33,12), 15:(34,12), 16:(35,12), 17:(36,12), 18:(37,12), 
                    19:(38,12), 20:(39,12), 21:(40,12), 22:(41,12), 23:(42,12), 24:(43,12),}}, 
    '応用化学変電所': {'sheetname': '日報2', 
           'data': {1:(20,13), 2:(21,13), 3:(22,13), 4:(23,13), 5:(24,13), 6:(25,13), 
                    7:(26,13), 8:(27,13), 9:(28,13), 10:(29,13), 11:(30,13), 12:(31,13), 
                    13:(32,13), 14:(33,13), 15:(34,13), 16:(35,13), 17:(36,13), 18:(37,13), 
                    19:(38,13), 20:(39,13), 21:(40,13), 22:(41,13), 23:(42,13), 24:(43,13),}}, 
    '事務局棟変電所': {'sheetname': '日報2', 
           'data': {1:(20,14), 2:(21,14), 3:(22,14), 4:(23,14), 5:(24,14), 6:(25,14), 
                    7:(26,14), 8:(27,14), 9:(28,14), 10:(29,14), 11:(30,14), 12:(31,14), 
                    13:(32,14), 14:(33,14), 15:(34,14), 16:(35,14), 17:(36,14), 18:(37,14), 
                    19:(38,14), 20:(39,14), 21:(40,14), 22:(41,14), 23:(42,14), 24:(43,14),}}, 
    'イノベーション変電所': {'sheetname': '日報2', 
           'data': {1:(20,15), 2:(21,15), 3:(22,15), 4:(23,15), 5:(24,15), 6:(25,15), 
                    7:(26,15), 8:(27,15), 9:(28,15), 10:(29,15), 11:(30,15), 12:(31,15), 
                    13:(32,15), 14:(33,15), 15:(34,15), 16:(35,15), 17:(36,15), 18:(37,15), 
                    19:(38,15), 20:(39,15), 21:(40,15), 22:(41,15), 23:(42,15), 24:(43,15),}}, 
    '教養学部変電所': {'sheetname': '日報2', 
           'data': {1:(20,16), 2:(21,16), 3:(22,16), 4:(23,16), 5:(24,16), 6:(25,16), 
                    7:(26,16), 8:(27,16), 9:(28,16), 10:(29,16), 11:(30,16), 12:(31,16), 
                    13:(32,16), 14:(33,16), 15:(34,16), 16:(35,16), 17:(36,16), 18:(37,16), 
                    19:(38,16), 20:(39,16), 21:(40,16), 22:(41,16), 23:(42,16), 24:(43,16),}}, 
    '理学部3号変電所': {'sheetname': '日報3', 
           'data': {1:(20,13), 2:(21,13), 3:(22,13), 4:(23,13), 5:(24,13), 6:(25,13), 
                    7:(26,13), 8:(27,13), 9:(28,13), 10:(29,13), 11:(30,13), 12:(31,13), 
                    13:(32,13), 14:(33,13), 15:(34,13), 16:(35,13), 17:(36,13), 18:(37,13), 
                    19:(38,13), 20:(39,13), 21:(40,13), 22:(41,13), 23:(42,13), 24:(43,13),}}, 
    '理学部2号変電所': {'sheetname': '日報3', 
           'data': {1:(20,14), 2:(21,14), 3:(22,14), 4:(23,14), 5:(24,14), 6:(25,14), 
                    7:(26,14), 8:(27,14), 9:(28,14), 10:(29,14), 11:(30,14), 12:(31,14), 
                    13:(32,14), 14:(33,14), 15:(34,14), 16:(35,14), 17:(36,14), 18:(37,14), 
                    19:(38,14), 20:(39,14), 21:(40,14), 22:(41,14), 23:(42,14), 24:(43,14),}}, 
    '電気電子1変電所': {'sheetname': '日報3', 
           'data': {1:(20,17), 2:(21,17), 3:(22,17), 4:(23,17), 5:(24,17), 6:(25,17), 
                    7:(26,17), 8:(27,17), 9:(28,17), 10:(29,17), 11:(30,17), 12:(31,17), 
                    13:(32,17), 14:(33,17), 15:(34,17), 16:(35,17), 17:(36,17), 18:(37,17), 
                    19:(38,17), 20:(39,17), 21:(40,17), 22:(41,17), 23:(42,17), 24:(43,17),}}, 
    '情報工学科変電所': {'sheetname': '日報3', 
           'data': {1:(20,19), 2:(21,19), 3:(22,19), 4:(23,19), 5:(24,19), 6:(25,19), 
                    7:(26,19), 8:(27,19), 9:(28,19), 10:(29,19), 11:(30,19), 12:(31,19), 
                    13:(32,19), 14:(33,19), 15:(34,19), 16:(35,19), 17:(36,19), 18:(37,19), 
                    19:(38,19), 20:(39,19), 21:(40,19), 22:(41,19), 23:(42,19), 24:(43,19),}}, 
    '分析センター変電所': {'sheetname': '日報4', 
           'data': {1:(20,4), 2:(21,4), 3:(22,4), 4:(23,4), 5:(24,4), 6:(25,4), 
                    7:(26,4), 8:(27,4), 9:(28,4), 10:(29,4), 11:(30,4), 12:(31,4), 
                    13:(32,4), 14:(33,4), 15:(34,4), 16:(35,4), 17:(36,4), 18:(37,4), 
                    19:(38,4), 20:(39,4), 21:(40,4), 22:(41,4), 23:(42,4), 24:(43,4),}}, 
    '大学会館変電所': {'sheetname': '日報4', 
           'data': {1:(20,10), 2:(21,10), 3:(22,10), 4:(23,10), 5:(24,10), 6:(25,10), 
                    7:(26,10), 8:(27,10), 9:(28,10), 10:(29,10), 11:(30,10), 12:(31,10), 
                    13:(32,10), 14:(33,10), 15:(34,10), 16:(35,10), 17:(36,10), 18:(37,10), 
                    19:(38,10), 20:(39,10), 21:(40,10), 22:(41,10), 23:(42,10), 24:(43,10),}}, 
    '図書館変電所': {'sheetname': '日報4', 
           'data': {1:(20,18), 2:(21,18), 3:(22,18), 4:(23,18), 5:(24,18), 6:(25,18), 
                    7:(26,18), 8:(27,18), 9:(28,18), 10:(29,18), 11:(30,18), 12:(31,18), 
                    13:(32,18), 14:(33,18), 15:(34,18), 16:(35,18), 17:(36,18), 18:(37,18), 
                    19:(38,18), 20:(39,18), 21:(40,18), 22:(41,18), 23:(42,18), 24:(43,18),}}, 
    '廃液処理変電所': {'sheetname': '日報4', 
           'data': {1:(20,19), 2:(21,19), 3:(22,19), 4:(23,19), 5:(24,19), 6:(25,19), 
                    7:(26,19), 8:(27,19), 9:(28,19), 10:(29,19), 11:(30,19), 12:(31,19), 
                    13:(32,19), 14:(33,19), 15:(34,19), 16:(35,19), 17:(36,19), 18:(37,19), 
                    19:(38,19), 20:(39,19), 21:(40,19), 22:(41,19), 23:(42,19), 24:(43,19),}}, 
    '教育学部C変電所': {'sheetname': '日報5', 
           'data': {1:(20,4), 2:(21,4), 3:(22,4), 4:(23,4), 5:(24,4), 6:(25,4), 
                    7:(26,4), 8:(27,4), 9:(28,4), 10:(29,4), 11:(30,4), 12:(31,4), 
                    13:(32,4), 14:(33,4), 15:(34,4), 16:(35,4), 17:(36,4), 18:(37,4), 
                    19:(38,4), 20:(39,4), 21:(40,4), 22:(41,4), 23:(42,4), 24:(43,4),}}, 
    '総合体育館変電所': {'sheetname': '日報5', 
           'data': {1:(20,8), 2:(21,8), 3:(22,8), 4:(23,8), 5:(24,8), 6:(25,8), 
                    7:(26,8), 8:(27,8), 9:(28,8), 10:(29,8), 11:(30,8), 12:(31,8), 
                    13:(32,8), 14:(33,8), 15:(34,8), 16:(35,8), 17:(36,8), 18:(37,8), 
                    19:(38,8), 20:(39,8), 21:(40,8), 22:(41,8), 23:(42,8), 24:(43,8),}}, 
    '経済学部変電所': {'sheetname': '日報5', 
           'data': {1:(20,9), 2:(21,9), 3:(22,9), 4:(23,9), 5:(24,9), 6:(25,9), 
                    7:(26,9), 8:(27,9), 9:(28,9), 10:(29,9), 11:(30,9), 12:(31,9), 
                    13:(32,9), 14:(33,9), 15:(34,9), 16:(35,9), 17:(36,9), 18:(37,9), 
                    19:(38,9), 20:(39,9), 21:(40,9), 22:(41,9), 23:(42,9), 24:(43,9),}}, 
    '総合研究棟変電所': {'sheetname': '日報5', 
           'data': {1:(20,11), 2:(21,11), 3:(22,11), 4:(23,11), 5:(24,11), 6:(25,11), 
                    7:(26,11), 8:(27,11), 9:(28,11), 10:(29,11), 11:(30,11), 12:(31,11), 
                    13:(32,11), 14:(33,11), 15:(34,11), 16:(35,11), 17:(36,11), 18:(37,11), 
                    19:(38,11), 20:(39,11), 21:(40,11), 22:(41,11), 23:(42,11), 24:(43,11),}}, 

                } # 


In [9]:
def get_data_from_excel(excel_files, output_file):
    print('# output to', output_file)
    # 集計先となるエクセルのブックを準備
    output_wb = openpyxl.Workbook()
    for key_1 in data_location:
        _total = 0
        # 元データのどのシートのどのデータを処理するか出力
        print('###', key_1, 'in', data_location[key_1]['sheetname'])
        # 出力用のワークブックに、ワークシートを作成して、出力用に準備
        output_wb.create_sheet(key_1)
        output_ws = output_wb[key_1] 
        # A列に項目名を入力
        output_ws.cell(row=1, column=1).value = 'Time'
        for i in range(24):
            output_ws.cell(row=2+i, column=1).value = i + 1
        output_column = 2
        output_ws.cell(row=26, column=1).value = 'Total'
        output_ws.cell(row=27, column=1).value = 'Peak'
        output_ws.cell(row=28, column=1).value = 'Day'
        output_ws.cell(row=29, column=1).value = 'Night'
        output_ws.cell(row=31, column=1).value = 'Max'

        # リスト中のエクセルファイルからデータを取得し、ワークシートに転記
        for excel_file in excel_files:
            # ファイル名から年月日を取得
            m = re.search(r'(202\d)(\d\d)(\d\d).*.xls', excel_file)
            if m is None:
                print(f'## unsupported file name', excel_file)
                return
            year = int(m.group(1))
            month = int(m.group(2))
            mday = int(m.group(3))

            # リスト中のエクセルファイルを開き、処理対象のワークシートがあるか確認
            # print(excel_file)
            wb = openpyxl.load_workbook(excel_file) # , ignore_workbook_corruption=True)
            # sheet_names = wb.sheet_names() # xlrd
            sheet_names = wb.sheetnames # openpyxl
            # print(excel_file, sheet_names) # [デバッグ用] エクセルファイル中のシート一覧を出力
            # 出力先シートに、データの年月日と、各時間帯の電力使用量の合計を計算する関数を挿入
            output_ws.cell(row=1, column=output_column).value = f'{year}{month:02d}{mday:02d}'
            output_ws.cell(row=26, column=output_column).value = f'=SUM({output_ws.cell(row=2, column=output_column).coordinate}:{output_ws.cell(row=25, column=output_column).coordinate})'
            output_ws.cell(row=27, column=output_column).value = f'=SUM({output_ws.cell(row=14, column=output_column).coordinate}:{output_ws.cell(row=16, column=output_column).coordinate})'
            output_ws.cell(row=28, column=output_column).value = f'=SUM({output_ws.cell(row=9, column=output_column).coordinate}:{output_ws.cell(row=13, column=output_column).coordinate})' + \
                                                                    f'+SUM({output_ws.cell(row=17, column=output_column).coordinate}:{output_ws.cell(row=23, column=output_column).coordinate})'
            output_ws.cell(row=29, column=output_column).value = f'=SUM({output_ws.cell(row=2, column=output_column).coordinate}:{output_ws.cell(row=8, column=output_column).coordinate})' + \
                                                                    f'+SUM({output_ws.cell(row=24, column=output_column).coordinate}:{output_ws.cell(row=25, column=output_column).coordinate})'
            output_ws.cell(row=31, column=output_column).value = f'=MAX({output_ws.cell(row=2, column=output_column).coordinate}:{output_ws.cell(row=25, column=output_column).coordinate})'
            if data_location[key_1]['sheetname'] not in sheet_names:
                print(f'### {year}/{month}/{mday} : sheet {data_location[key_1]["sheetname"]} is not in {excel_file}')
            else:
                # リスト中のエクセルファイルからデータを出力先シートに転記
                # sheet = wb.sheet_by_name(data_location[key_1]['sheetname']) # xlrd
                sheet = wb[data_location[key_1]['sheetname']] # openpyxl
                output_row = 2
                for key_2 in data_location[key_1]['data'].keys():
                    _row, _column = data_location[key_1]['data'][key_2] 
                    # 数値(int, float)または空白のセルを出力先シートに転記
                    if sheet.cell(_row, _column).value == '' or isinstance(sheet.cell(_row, _column).value, int) or isinstance(sheet.cell(_row, _column).value, float):
                        output_ws.cell(row=output_row, column=output_column).value = sheet.cell(_row, _column).value
                        if isinstance(sheet.cell(_row, _column).value, float) or isinstance(sheet.cell(_row, _column).value, int):
                            _total = _total + float(sheet.cell(_row, _column).value)
                    output_row = output_row + 1
            output_column = output_column + 1
        print(f'total: {_total}')
    
    # ワークブックを出力 (空のシートは削除)
    output_wb.remove(output_wb['Sheet'])
    output_wb.save(output_file)

In [10]:
# 電力データを保存したフォルダにあるエクセルファイルのリストを作成
data_folder = r'E:\sudspg\資料\電力使用量_2024-2' # 電力データを保存したフォルダ名 (旧フォーマット)
data_files = sorted(glob.glob(os.path.join(data_folder, '202*.xlsx'))) # エクセルファイル名のリストを取得
print(f'# {len(data_files)} files')
if len(data_files) == 0:
    print('## no data files')
    sys.exit(1)
print('data_file:', data_files[0], ' ～ ', data_files[-1])

# リスト中のエクセルファイルを集約したエクセルファイルを作成
get_data_from_excel(data_files, 'dm04_sample_data_2024-2.xlsx')

# 90 files
data_file: E:\sudspg\資料\電力使用量_2024-2\20250101_R.xlsx  ～  E:\sudspg\資料\電力使用量_2024-2\20250331_R.xlsx
# output to dm04_sample_data_2024-2.xlsx
### 全体 in 日報1
total: 3085860.0
### 電気電子3変電所 in 日報2
total: 101480.0
### 機械工学変電所 in 日報2
total: 113780.0
### デザイン1号変電所 in 日報2
total: 291730.0
### 応用化学変電所 in 日報2
total: 154250.0
### 事務局棟変電所 in 日報2
total: 42110.0
### イノベーション変電所 in 日報2
total: 46500.0
### 教養学部変電所 in 日報2
total: 30000.0
### 理学部3号変電所 in 日報3
total: 594040.0
### 理学部2号変電所 in 日報3
total: 98070.0
### 電気電子1変電所 in 日報3
total: 156230.0
### 情報工学科変電所 in 日報3
total: 21360.0
### 分析センター変電所 in 日報4
total: 114980.0
### 大学会館変電所 in 日報4
total: 104420.0
### 図書館変電所 in 日報4
total: 49020.0
### 廃液処理変電所 in 日報4
total: 4430.0
### 教育学部C変電所 in 日報5
total: 282000.0
### 総合体育館変電所 in 日報5
total: 41580.0
### 経済学部変電所 in 日報5
total: 37380.0
### 総合研究棟変電所 in 日報5
total: 227490.0
